In [1]:
import pandas as pd

In [5]:
Churn_data=pd.read_excel('Telecom Churn Rate Dataset.xlsx')

In [6]:
Churn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [8]:
Churn_data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,numAdminTickets,numTechTickets,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,0,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,One year,No,Mailed check,56.95,1889.5,0,0,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,0,0,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,3,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,0,0,Yes


- customerID: 고객 식별번호
- gender: 고객 성별
- SeniorCitizen: 노년 고객 여부 (1: 노년고객, 0: 노년고객 아님)
- Partner: 고객의 파트너 여부
- Dependents: 고객의 부양가족 여부
- tenure: 고객이 사용한 달 수
- PhoneService: 전화 서비스 사용 여부
- MultipleLines: 다중 전화 회선 사용 여부
- InternetService: 인터넷 서비스 종류
- OnlineSecurity: 온라인 보안 여부
- OnlineBackup: 온라인 백업 여부
- DeviceProtection: 기기 보호 여부
- TechSupport: 기술 지원 여부
- StreamingTV: 스트리밍 TV 사용 여부
- StreamingMovies: 스트리밍 영화 사용 여부
- Contract: 계약 기간
- PaperlessBilling: 종이 청구서 수신 여부
- PaymentMethod: 결제 수단
- MonthlyCharges: 월별 요금
- TotalCharges: 총 요금
- numAdminTickets: 관리티켓 갯수
- numTechTickets: 기술티켓 갯수
- Churn: 이탈 여부 (Yes: 이탈, No: 이탈하지 않음)